<a href="https://colab.research.google.com/github/Yuweien/Python-Workshop/blob/main/Python_Workshop_for_Beginners_2_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Programming: Reading, Understanding, and Modifying Code — Part Two

## Current workflow roadmap

**✅ Raw D2L CSV**  
&nbsp;&nbsp;&nbsp;&nbsp;↓  
**🔵 Inspect & clean**  
&nbsp;&nbsp;&nbsp;&nbsp;↓  
**⬜ Add group info (merge)**  
&nbsp;&nbsp;&nbsp;&nbsp;↓  
**⬜ Split by question type**  
&nbsp;&nbsp;&nbsp;&nbsp;├── **⬜ MC** → group stats → bar chart  
&nbsp;&nbsp;&nbsp;&nbsp;└── **⬜ Short answer** → word freq → word cloud



# Step 0. Files and setup

### You are here
- 🔵 **Raw D2L CSV**
- ⬜ Inspect & clean
- ⬜ Add group info (merge)
- ⬜ Split by question type

In this step, we:
- Load the raw D2L CSV file
- Take a first look at what the data looks like

👉 Goal: *Understand what we are working with before touching the data.*



In [1]:
# code goes here


# Step 1. Inspect & clean data

### Workflow status
- ✅ Raw D2L CSV
- 🔵 **Inspect & clean**
- ⬜ Add group info (merge)
- ⬜ Split by question type

In this step, we:
- Inspect columns and basic structure
- Clean obvious issues (extra spaces, column names, unnecessary columns)

👉 Goal: *Make the data reliable for later steps.*



In [ ]:
# code goes here


# Step 2. Add group information (merge)

### Workflow status
- ✅ Raw D2L CSV
- ✅ Inspect & clean
- 🔵 **Add group info (merge)**
- ⬜ Split by question type

In this step, we:
- Load a separate group roster
- Merge group information into the main dataset

👉 Goal: *Add meaningful context (groups) to the data.*



In [ ]:
# code goes here


# Step 3. Split by question type

### Workflow status
- ✅ Raw D2L CSV
- ✅ Inspect & clean
- ✅ Add group info (merge)
- 🔵 **Split by question type**

In this step, we:
- Separate multiple-choice questions from short-answer questions
- Prepare different analysis paths for different data types

👉 Goal: *Different data types need different analysis strategies.*



In [ ]:
# code goes here


# Step 4A. Multiple-choice questions: group stats & visualization

### Workflow status
- ✅ Raw D2L CSV
- ✅ Inspect & clean
- ✅ Add group info (merge)
- ✅ Split by question type
  - 🔵 **MC → group stats → bar chart**
  - ⬜ Short answer → word freq → word cloud

In this step, we:
- Calculate simple statistics by group
- Create a basic bar chart to compare groups

👉 Goal: *Use simple statistics to answer a teaching or research question.*



In [ ]:
# code goes here


# Step 4B. Short-answer questions: text exploration (optional)

### Workflow status
- ✅ Raw D2L CSV
- ✅ Inspect & clean
- ✅ Add group info (merge)
- ✅ Split by question type
  - ⬜ MC → group stats → bar chart
  - 🔵 **Short answer → word freq → word cloud**

In this step, we:
- Explore common words or phrases in open-ended responses
- Visualize themes using word frequency or a word cloud

👉 Goal: *Get a quick, exploratory sense of what students are saying.*

*(Optional — skip if time is limited.)*



# Wrap-up: Adapting this workflow to your own data

We’ve walked through a complete workflow:
- From raw LMS data
- To cleaned, combined, and analyzed results

Think about:
- Which steps are essential for your own project?
- Where might you stop, simplify, or extend the workflow?
- How could AI tools help you modify this code safely?

👉 The goal is not to memorize code,  
but to **read, understand, and adapt workflows**.
